<a href="https://colab.research.google.com/github/Avichay3/final_project_eyal/blob/main/RoberTa_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q transformers datasets accelerate evaluate scikit-learn




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [4]:
import transformers
print(transformers.__version__)


4.57.1


In [5]:
from google.colab import files
uploaded = files.upload()


Saving heldout_pair_data.jsonlist to heldout_pair_data.jsonlist


In [14]:
import json, pandas as pd, re

path = "heldout_pair_data.jsonlist"

def clean_text(t):
    t = re.sub(r'\n+', ' ', t)
    t = re.sub(r'\s+', ' ', t)
    t = re.sub(r'http\S+', '', t)
    t = re.sub(r'\[.*?\]\(.*?\)', '', t)  # הסרת markdown
    return t.strip()

data = []
with open(path, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        for group, label in [("positive", 1), ("negative", 0)]:
            if group in obj and "comments" in obj[group]:
                for c in obj[group]["comments"]:
                    text = clean_text(c.get("body", ""))
                    if text:
                        data.append({"text": text, "label": label})

df = pd.DataFrame(data)
print("✅ Loaded:", len(df), "samples")
print(df["label"].value_counts(normalize=True))


✅ Loaded: 2142 samples
label
1    0.529412
0    0.470588
Name: proportion, dtype: float64


In [42]:
from transformers import AutoTokenizer

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)

tokenized = dataset.map(tokenize, batched=True)
tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/1713 [00:00<?, ? examples/s]

Map:   0%|          | 0/214 [00:00<?, ? examples/s]

Map:   0%|          | 0/215 [00:00<?, ? examples/s]

In [43]:
from transformers import AutoTokenizer

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)

tokenized = dataset.map(tokenize, batched=True)
tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])


Map:   0%|          | 0/1713 [00:00<?, ? examples/s]

Map:   0%|          | 0/214 [00:00<?, ? examples/s]

Map:   0%|          | 0/215 [00:00<?, ? examples/s]

In [44]:
from transformers import AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average="binary")
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


In [46]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
import os

os.environ["WANDB_DISABLED"] = "true"
use_fp16 = torch.cuda.is_available()

args = TrainingArguments(
    output_dir="./persuasion_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1.5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.05,
    warmup_ratio=0.1,
    fp16=use_fp16,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=2,
    logging_steps=100,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()


/tmp/ipython-input-3172737306.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.700700,0.702492,0.518692,0.524038,0.964602,0.679128
2,0.686400,0.697938,0.528037,0.538961,0.734513,0.621723
3,0.672400,0.687960,0.528037,0.560000,0.495575,0.525822


TrainOutput(global_step=645, training_loss=0.6790751139322917, metrics={'train_runtime': 155.5413, 'train_samples_per_second': 55.066, 'train_steps_per_second': 6.911, 'total_flos': 1352127713495040.0, 'train_loss': 0.6790751139322917, 'epoch': 3.0})

In [48]:
print("\n✅ Validation Results:")
print(trainer.evaluate(tokenized["validation"]))

print("\n✅ Test Results:")
print(trainer.evaluate(tokenized["test"]))



✅ Validation Results:


{'eval_loss': 0.702491819858551, 'eval_accuracy': 0.5186915887850467, 'eval_precision': 0.5240384615384616, 'eval_recall': 0.9646017699115044, 'eval_f1': 0.6791277258566978, 'eval_runtime': 1.5877, 'eval_samples_per_second': 134.782, 'eval_steps_per_second': 17.005, 'epoch': 3.0}

✅ Test Results:
{'eval_loss': 0.6864950656890869, 'eval_accuracy': 0.5302325581395348, 'eval_precision': 0.5317073170731708, 'eval_recall': 0.956140350877193, 'eval_f1': 0.6833855799373041, 'eval_runtime': 1.5022, 'eval_samples_per_second': 143.126, 'eval_steps_per_second': 17.974, 'epoch': 3.0}
